# 4. LangChain解説

In [ ]:
import os


os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

## 4-1 LangChainの概要

In [ ]:
!pip install --quiet langchain openai

## 4-2 Language models

### LLMs

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

result = llm("自己紹介してください。")
print(result)

### Chat models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)

### Callbackを使ったストリーミング

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages)

## 4-3 Prompts

### Prompt templates

In [ ]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを教えてください。

料理名: {dish}
"""

prompt = PromptTemplate(
   input_variables=["dish"],
   template=template,
)

result = prompt.format(dish="カレー")
print(result)

### ChatPromptTemplate

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
    HumanMessagePromptTemplate.from_template("以下の料理のレシピを教えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_prompt(country="フランス", dish="肉じゃが").to_messages()

print(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat(messages)
print(result.content)

## 4-4 Output Parsers


### PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
from logging import Formatter
format_instructions = parser.get_format_instructions()

print(format_instructions)

In [ ]:
from langchain.prompts import PromptTemplate

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

In [ ]:
input = prompt.format(dish="カレー")

print(input)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=input)]
output = chat(messages)

print(output.content)

In [ ]:
recipe = parser.parse(output.content)
print(recipe)

## 4-4 Chains

### LLMChain―PromptTemplate・Language model・OutputParserをつなぐ

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.run(dish="カレー")

print(type(recipe))
print(recipe)

### SimpleSequentialChain―ChainとChainをつなぐ

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

cot_template = """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [ ]:
summarize_template = """以下の文章を結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain(
   "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])

## 4-5 Memory

### ConversationBufferMemory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

import langchain
langchain.debug = True
langchain.verbose = False

chat = ChatOpenAI(model_name="gpt-4", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)